# Utilisation avec JupyterLite (version web)

 - Après avoir ouvert le notebook (donc si tu vois ça), il faut attendre un peu que tout démarre bien
 - En haut à droite, il y a un petit rond à côté de "Python (Pyodide)", qui peut être remplit en gris ou avec un petit éclair. En glissant le curseur dessus, il y a "Kernel status: Unknown/Idle/Busy". Attendre que le rond ne soit plus remplit et "Kernel status: Idle" soit écrit.
 - Maintenant, aller sur Run > Run all cells, et l'interface avec les boutons devrait s'afficher tout en bas.
 - Au cas ou quelque chose ne marche pas, cliquer sur la petite flèche en boucle ("Restart the kernel"), vérifier que le bouton indique bien "Python (Pyodide)" (si c'est "No Kernel", cliquer dessus, sélectionner "Python (Pyodide)" et confirmer) puis attendre que le rond ne soit plus remplit, puis réessayer.

In [ ]:
print("Si ce message est affiche, c'est qu'on execute au moins du code.")

In [ ]:
%pip install openpyxl nbformat plotly dash widgetsnbextension~=4.0.10 ipywidgets==8.1.2

In [ ]:
import numpy as np
from scipy.integrate import cumulative_trapezoid
from scipy.signal import find_peaks
from scipy import signal
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, callback
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from ipywidgets import widgets
from dataclasses import dataclass
from typing import TextIO
import io
import re

In [ ]:
@dataclass
class TestFile:
    excel_file: bytes | None
    excel_path: Path
    raw_file: bytes | None
    raw_path: Path
    label: str

def list_test_file_pairs(data_dir: Path) -> list[TestFile]:
    # Todo update this logic for files with name as prefix
    files: list[TestFile] = []
    for xls in data_dir.glob('*.xlsx'):
        prefix = xls.name.split('_')[0]
        log_match = list(data_dir.glob(f'{prefix}*raw.log'))
        if len(log_match) == 0:
            print(f'No matching raw log for Excel file {xls}')
        elif len(log_match) > 1:
            print(f'More then 1 matching raw log for Excel file {xls}?')
        else:
            files.append(TestFile(xls, log_match[0], label=prefix))
    return files

file_number_re = re.compile(r'\d+_\d{1,2}_\d{1,2}_\d\d\d\d')
def group_files(uploaded_files) -> list[TestFile]:
    raw_files = dict()
    excel_files = dict()
    for f in uploaded_files:
        match = file_number_re.search(f.name)
        if match:
            fid = match.group()
            if f.name.endswith('.xlsx'):
                excel_files[fid] = f
            elif f.name.endswith('.log'):
                raw_files[fid] = f
            else:
                print(f'Warning: filename {f.name} not in expected format')
        else:
            print(f'Warning: filename {f.name} not in expected format')
    files = []
    for fid, xls in excel_files.items():
        if not fid in raw_files:
            print(f'Missing raw log file for {xls.name}')
            continue
        excel_file = xls.content.tobytes()
        rf = raw_files[fid]
        raw_file = rf.content.tobytes()
        files.append(TestFile(excel_file, Path(xls.name), raw_file, Path(rf.name), label=fid))
    return files

In [ ]:
cycle_col_names = ['phase', 'load', 'vo2/kg', 'fc', 'vo2', 'vco2', 'qr', 'vol_instant', 'bf', 've', 've/vo2', 've/vco2', 'peto2', 'petco2',
    'vol_in', 't_in', 'vol_ex', 't_ex', 'pulse_o2', 'spo2', 'sbp', 'dbp', 'rpm', 'fo2et', 'fco2et', 'fio2et', 'fico2', 'feo2', 'feco2', 'delay_o2', 'delay_co2', 'temp_ambient',
    'pressure_ambient', 'humidity_ambient', 'duration']
def load_dataframes(file: TestFile, debug=False):
    print(f'Loading files:\n{str(file.excel_path)}\n{str(file.raw_path)}')
    # some xls files have empty rows at the top, so read once to know how many to skip
    df_cycles = pd.read_excel(io.BytesIO(file.excel_file), header=None)

    drop_cols = [1]
    keep_cols = [col for col in range(len(df_cycles.columns)) if col not in drop_cols]
    assert len(keep_cols) == len(cycle_col_names)
    # drop all columns except indices in keep_cols
    df_cycles.drop([df_cycles.columns[col] for col in drop_cols], axis=1, inplace=True)
    df_cycles.columns = cycle_col_names
    df_cycles['phase'] = df_cycles['phase'].replace('Repos', 'rest')
    df_cycles['phase'] = df_cycles['phase'].replace('Charge', 'load')
    df_cycles['phase'] = df_cycles['phase'].replace('Récupération', 'recovery')
    df_cycles['phase'] = df_cycles['phase'].ffill()
    # for all columns except col 0, cast to float and set string data in headers to NaN
    for col in df_cycles.columns[1:]:
        df_cycles[col] = pd.to_numeric(df_cycles[col], errors='coerce')
    first_valid_row = 0
    while df_cycles.loc[first_valid_row, ['vol_instant', 'vol_in', 'vol_ex', 't_in', 't_ex', 'duration']].isna().any():
        first_valid_row += 1
    last_valid_row = df_cycles.count().max()
    if debug: print(f'Dropping {first_valid_row - 1} first rows')
    df_cycles.drop(range(first_valid_row), inplace=True)
    df_cycles.reset_index(drop=True, inplace=True)
    n_rows = len(df_cycles)
    invalid_rows_at_end = 0
    while df_cycles.loc[n_rows - invalid_rows_at_end - 1, df_cycles.columns[1:]].isna().any():
        invalid_rows_at_end += 1
    if debug: print(f'Dropping {invalid_rows_at_end} last rows of {n_rows}')
    if invalid_rows_at_end:
        df_cycles.drop(range(n_rows - invalid_rows_at_end, n_rows), inplace=True)

    df_cycles.reset_index(drop=True, inplace=True)
    if debug: print(f'{len(df_cycles)} cycle rows')

    df_raw = pd.read_csv(io.BytesIO(file.raw_file), delimiter='\t', names=['t', 'flow', 'fo2', 'fco2'])
    first_non_zero_flow_row = 0
    flow_thresh = 0.2
    flow_almost_zero = df_raw[df_raw['flow'].abs() > flow_thresh]
    if len(flow_almost_zero) > 0:
        # start a couple of samples before flow takes >0 values
        flow_almost_zero_cutoff = max(0, flow_almost_zero.index[0] - 300) 
        if flow_almost_zero_cutoff > 0:
            if debug: print(f'Dropping first {flow_almost_zero_cutoff} rows with flow<{flow_thresh}')
            df_raw.drop(range(flow_almost_zero_cutoff), inplace=True)
    df_raw.reset_index(inplace=True, drop=True)
    df_raw['t'] = df_raw['t'] - df_raw.loc[0, 't']
    return df_cycles, df_raw

@dataclass
class CorrResult:
    winstart: int
    winend: int
    shift: int
    win_duration_diffs: np.ndarray
    win_durations_cycles: np.ndarray
    corrs: np.ndarray
    good_match: bool

def find_best_corr_window(df_cycles, durations_hires, debug=False) -> CorrResult | None:
    want_winsize = 100
    best_winstart, best_winend, best_mean_err, best_shift = None, None, None, None
    good_match = False
    winstart = min(20, len(df_cycles))
    winend = min(winstart + want_winsize, len(df_cycles))

    # often times there is a stretch of zeros a few seconds into the data, and we want to cut that off.
    MAX_SANE_CYCLE_DURATION = 30 # seconds, max duration a cycle could possibly be
    abnormal_long_cycles_hires = durations_hires > MAX_SANE_CYCLE_DURATION
    normal_cycles_hires = np.logical_not(abnormal_long_cycles_hires)
    if not normal_cycles_hires.any():
        print(f'All hires cycle longer than {MAX_SANE_CYCLE_DURATION}, this can not be right. Aborting')
        return None
    # obviously at the very end of data there is very long/invalid cycles which we don't want to consider for cutting off
    abnormal_long_cycles_hires[normal_cycles_hires.argmax() - 50:] = False
    start_cutoff_hires = min(100, abnormal_long_cycles_hires.argmax()) if abnormal_long_cycles_hires.any() else 30 # cut off first hires samples that may have extreme values
    if debug:
        print(f'Cutting off {start_cutoff_hires} invalid/very long cycles at the beginning')
    best_mean_err = None
    while not good_match and winend <= len(df_cycles):
        winsize = winend - winstart
        if winsize <= 0:
            print(f'Could not find any correlation')
            return None
        if len(durations_hires) < winsize:
            print(f'Error: not enough cycles in raw data to match window of size {winsize}')
            return None
        if debug: print(f'Finding correlation over window {winstart}-{winend}')
        if winsize < 30:
            print(f'Warning: small window to find initial cycle correlation')
        # durations_hires[i] is duration of cycle from valls[i] to valls[i+1]
        win_durations_cycles = df_cycles.loc[range(winstart, winend), 'duration'].array
        corrs = np.correlate(durations_hires[start_cutoff_hires:] - durations_hires.mean(), win_durations_cycles - win_durations_cycles.mean(), mode='valid')
        shift = corrs.argmax() + start_cutoff_hires

        duration_diffs = df_cycles.loc[range(winstart, winend), 'duration'].array - durations_hires[shift:shift+winsize]
        mean_err = np.abs(duration_diffs).mean()
        if debug: print(f'Mean cycle duration error in {winsize} cycle window: {mean_err}')
        if best_mean_err is None or mean_err < best_mean_err:
            best_winstart = winstart
            best_winend = winend
            best_mean_err = mean_err
            best_shift = shift
            duration_diffs_sorted = np.sort(duration_diffs)
            # sort and throw away worst 3 values when deciding if match is good or not
            good_match = np.abs(duration_diffs_sorted[:-3]).mean() < 0.2
        if not good_match:
            print(f'Bad correlation in window {winstart}-{winend}')
        winstart += 30
        winend = min(winstart + want_winsize, len(df_cycles))
    assert best_winstart is not None
    assert best_winend is not None
    assert best_shift is not None
    if good_match:
        print(f'Good correlation in window {winstart}-{winend}')
    win_durations_cycles = df_cycles.loc[range(best_winstart, best_winend), 'duration'].array
    winsize = best_winend - best_winstart
    win_duration_diffs = df_cycles.loc[range(best_winstart, best_winend), 'duration'].array - durations_hires[best_shift:best_shift+winsize]
    return CorrResult(best_winstart, best_winend, best_shift, win_duration_diffs, win_durations_cycles, corrs, good_match)

def match_cycles_with_raw_data(df_cycles, df_raw, output, debug = False) -> bool:
    sampling_freq_hz = 1 / (df_raw['t'][:-1] - df_raw['t'].shift(1)[1:]).mean()
    filter_freq_hz = 3.15*1e-2
    df_raw['instant_vol_raw'] = cumulative_trapezoid(y=df_raw['flow'], x=df_raw['t'], initial=0) 
    sos = signal.butter(4, Wn=filter_freq_hz / sampling_freq_hz, btype='highpass', output='sos')
    flow_filtered = signal.sosfilt(sos, df_raw['flow'])
    df_raw['instant_vol'] = cumulative_trapezoid(y=flow_filtered, x=df_raw['t'], initial=0) 
    if debug: print(f'Sum over all instantaneous volume: {df_raw.instant_vol.sum()}')

    MIN_PROMINENCE = 0.15
    peaks, peakprops  = signal.find_peaks(df_raw['instant_vol'], prominence=MIN_PROMINENCE)
    valls, vallprops = signal.find_peaks(-df_raw['instant_vol'], prominence=MIN_PROMINENCE)
    if debug: print(f'Found {len(peaks)} peaks, {len(valls)} valleys')
    # index of first peak that comes after first valley
    first_peak_idx = np.argwhere(peaks > valls[0]).min()
    if not (valls[-1] > peaks).all(): # there is a peak after the last valley
        # index of last peak that comes before last valley
        last_peak_idx = np.argwhere(valls[-1] < peaks).min()
    else:
        last_peak_idx = len(peaks - 1)
    if debug: print(f'Dropping first {first_peak_idx} and last {len(peaks) - last_peak_idx} peaks')
    peaks = peaks[first_peak_idx:last_peak_idx]

    #   p   p   p   n peaks
    #  / \_/ \_/ \
    # v   v   v   v n+1 valleys

    # That makes n complete cycles

    assert len(peaks) == len(valls) - 1
    iv = df_raw['instant_vol']
    vins = iv[peaks].array - iv[valls[:-1]].array
    vexs = iv[peaks].array - iv[valls[1:]].array
    if debug: print(f'Found {len(peaks)} complete cycles in 125Hz data')


    # durations_hires[i] is duration of cycle from valls[i] to valls[i+1]
    durations_hires = df_raw['t'][valls[1:]].array - df_raw['t'][valls[:-1]].array

    corr_result = find_best_corr_window(df_cycles, durations_hires, debug)
    if corr_result is None:
        print(f'Error: Could not find correlation to align high-res and cycle by cycle data')
        return False
    shift = corr_result.shift
    winstart, winend = corr_result.winstart, corr_result.winend
    if not corr_result.good_match:
        print(f'Warning: potentially bad match between cycles and 125Hz data')
    if debug or not corr_result.good_match:
        fig, axs = plt.subplots(2)
        axs[0].bar(range(len(corr_result.corrs)), corr_result.corrs), shift
        winsize = winend - winstart
        axs[0].set_title(f'Correlation for window of {winsize} cycles (raw data)')

        axs[1].plot(range(winsize), durations_hires[shift:shift+winsize], label='cycle duration 125Hz')
        axs[1].plot(range(winsize), corr_result.win_durations_cycles, label='cycle duration')
        # axs[1].plot(range(winsize), df_cycles.loc[range(winstart, winend), 'duration'], label='cycle duration')
        axs[1].legend()
        with output:
            display(fig)

    best_duration_match_idx = np.abs(corr_result.win_duration_diffs).argmin()

    df_raw['cycle_index'] = pd.Series(dtype=int)
    df_cycles['hires_tstart'] = pd.Series(dtype=float)
    df_cycles['hires_tend'] = pd.Series(dtype=float)
    df_cycles['hires_mismatch'] = pd.Series(dtype=bool)
    df_cycles['hires_mismatch'] = False
    matched_cycle_index = winstart + best_duration_match_idx
    matched_hires_valley_idx = shift + best_duration_match_idx
    if debug: print(f'Matched cycle {matched_cycle_index}')
    df_cycles.loc[matched_cycle_index, 'hires_tstart'] = df_raw.loc[valls[matched_hires_valley_idx], 't']
    # df_cycles.loc[matched_cycle_index, 'hires_tend'] = df_raw.loc[valls[matched_hires_valley_idx + 1], 't']

    d = df_raw.loc[valls[matched_hires_valley_idx], 't'] - df_raw.loc[valls[matched_hires_valley_idx - 1], 't']
    dd = df_cycles.loc[matched_cycle_index, 't_in'] + df_cycles.loc[matched_cycle_index, 't_ex']
    df_cycles.loc[matched_cycle_index]
    # accepted relative error (%) between durations from high-res and cycle-by-cycle data
    MAX_DURATION_ERROR = 20 / 100

    # last matched valley index. start of cycle after this one in time
    current_valley_idx = matched_hires_valley_idx
    # walk backwards in time, matching up cycles before matched_cycle_index
    for cycle_idx in reversed(range(0, matched_cycle_index)):
        cycle_tend = df_cycles.loc[cycle_idx + 1, 'hires_tstart']
        df_cycles.loc[cycle_idx, 'hires_tend'] = cycle_tend
        if current_valley_idx <= 0:
            print(f'Warning: not enough cycles in raw data ({cycle_idx+1} cycles left to match up during backwards walk, but no more local minima in 125Hz data)')
            break
        valls_before = valls[:current_valley_idx]
        # duration if cycle starts at a valley 
        duration_valley_start = -df_raw.loc[valls_before, 't'].array + cycle_tend
        true_cycle_duration = df_cycles.loc[cycle_idx, 't_in'] + df_cycles.loc[cycle_idx, 't_ex']
        best_valley_idx = np.argmin(np.abs(duration_valley_start - true_cycle_duration))
        duration_error = (duration_valley_start[best_valley_idx] - true_cycle_duration) / duration_valley_start[best_valley_idx]
        if np.abs(duration_error) < MAX_DURATION_ERROR:
            df_cycles.loc[cycle_idx, 'hires_tstart'] = df_raw.loc[valls_before[best_valley_idx], 't']
            current_valley_idx = best_valley_idx
        else: # no valley matches cycle duration in excel data
            # duration if cycle starts at any t
            duration_t = -df_raw['t'].array + cycle_tend
            best_raw_idx = np.argmin(np.abs(duration_t - true_cycle_duration))
            while current_valley_idx >= 0 and best_raw_idx <= valls[current_valley_idx]: 
                current_valley_idx -= 1
            # must point to last matched valley, which does not exist here. +1 b/c otherwise we lose 1 valley in valls_before = valls[:current_valley_idx] 
            current_valley_idx += 1 
            df_cycles.loc[cycle_idx, 'hires_tstart'] = df_raw.loc[best_raw_idx, 't']
            df_cycles.loc[cycle_idx, 'hires_mismatch'] = True
    
    current_valley_idx = matched_hires_valley_idx + 1
    # walk forwards in time, matching up cycles after matched_cycle_index
    for cycle_idx in range(matched_cycle_index, len(df_cycles)):
        cycle_tstart = df_cycles.loc[cycle_idx - 1, 'hires_tend']
        df_cycles.loc[cycle_idx, 'hires_tstart'] = cycle_tstart
        if current_valley_idx > len(valls) - 1:
            print(f'Warning: not enough cycles in raw data ({len(df_cycles)-cycle_idx-1} cycles left to match up during forwards walk, but no more local minima in 125Hz data)')
            break
        valls_after = valls[current_valley_idx:]
        true_cycle_duration = df_cycles.loc[cycle_idx, 't_in'] + df_cycles.loc[cycle_idx, 't_ex']
        # duration if cycle ends at a valley 
        duration_valley_end = df_raw.loc[valls_after, 't'].array - cycle_tstart
        best_valley_idx = np.argmin(np.abs(duration_valley_end - true_cycle_duration))
        duration_error = np.abs(duration_valley_end[best_valley_idx] - true_cycle_duration) / true_cycle_duration
        if duration_error < MAX_DURATION_ERROR:
            df_cycles.loc[cycle_idx, 'hires_tend'] = df_raw.loc[valls_after[best_valley_idx], 't']
            current_valley_idx += best_valley_idx + 1 # best_valley_idx indexes into the slide valls_after so it's an offset on top of current_valley_idx
            continue
        else:
            pass
            # print(f'cycle {cycle_idx} duration error {duration_error}')
        # no valley matched
        # duration if cycle starts at any t
        raw_after_last_matched = df_raw.loc[range(valls[current_valley_idx], len(df_raw))]
        duration_t = raw_after_last_matched['t'] - cycle_tstart
        best_raw_idx = (duration_t - true_cycle_duration).idxmin()
        while current_valley_idx < len(valls) and valls[current_valley_idx] <= best_raw_idx:
            current_valley_idx += 1
        df_cycles.loc[cycle_idx, 'hires_tend'] = df_raw.loc[best_raw_idx, 't']
        df_cycles.loc[cycle_idx, 'hires_mismatch'] = True

    df_cycles['hires_duration'] = df_cycles['hires_tend'] - df_cycles['hires_tstart']
    for index, cycle in df_cycles.iterrows():
        df_raw.loc[(cycle['hires_tstart'] <= df_raw['t']) & (df_raw['t'] < cycle['hires_tend']), 'cycle_index'] = index
    return True

def find_sighs(df_cycles, window_size: int, vol='avg', center: bool=True):
    if vol == 'avg':
        col = 'vol_avg'
        df_cycles['vol_avg'] = 0.5 * (df_cycles['vol_in'] + df_cycles['vol_ex'])
    if vol == 'in': col = 'vol_in'
    if vol == 'ex': col = 'vol_ex'
    vol = df_cycles[col]
    col_rolling = f'{col}_rolling_median'
    df_cycles[col_rolling] = vol.rolling(window=window_size, center=center).median().bfill().ffill()
    df_cycles['is_sigh'] = vol > 2 * df_cycles[col_rolling]

In [ ]:
def on_analyze_clicked(file: TestFile, output, sigh_vol='avg', rolling_center: bool=True, debug=False):
    global df_cycles
    global df_raw
    global test_file 
    test_file = file
    df_cycles, df_raw = load_dataframes(file, debug)
    if not match_cycles_with_raw_data(df_cycles, df_raw, output, debug):
        return
    find_sighs(df_cycles, window_size=15, vol=sigh_vol, center=rolling_center)
    
    df_raw['ts'] = pd.to_datetime(df_raw['t'], unit='s')
    cycle_maxs = df_raw.dropna(subset='cycle_index').sort_values('instant_vol', ascending=False).drop_duplicates('cycle_index').sort_values('cycle_index')
    cycle_maxs = cycle_maxs.join(df_cycles, on='cycle_index')
    fig = px.line(df_raw, y=['instant_vol', 'flow'], x='ts', height=500)
    names={'flow': 'Flow', 'instant_vol': 'Vol. Instant.'}
    fig.for_each_trace(lambda t: t.update(name = names[t.name],
                                      legendgroup = names[t.name]
                                         )
                  )
    for tstart in df_cycles.loc[df_cycles['is_sigh'],'hires_tstart']:
        fig.add_vline(x=pd.to_datetime(tstart, unit='s'))
    def hover_text(df) -> str:
        cols_labels = {
            'vol_in': 'Vol ins',
            'vol_ex': 'Vol exp',
            't_in': 'Durée ins',
            't_ex': 'Durée exp',
            'load': 'Charge',
        }
        fields = '<br />'.join([f'{label}: {df[col]}' for col, label in cols_labels.items()])
        msg = f'Cycle {int(df["cycle_index"])}<br />{fields}'
        return msg
    cycle_maxs['hover_text'] = cycle_maxs.apply(hover_text, axis=1)
    scatter = go.Scatter(
            x=cycle_maxs['ts'],
            y=cycle_maxs['instant_vol'],
            text=cycle_maxs['hover_text'],
            name='Cycles',
            mode='markers',
        )
    fig.add_trace(scatter)
    fig.update_layout(xaxis_tickformat='%H:%M:%S',
        legend_title_text='',
        xaxis_title="Time",
        yaxis_title="Volume (L)")

    phase_changes = list(df_cycles.loc[df_cycles['phase'].shift(1) != df_cycles['phase']].iterrows())
    text_y = df_raw['instant_vol'].max()
    phase_colors = {
        'rest': 'LightGreen', 
        'recovery': 'LightGreen', 
        'load': 'LightSkyBlue'
    }
    phase_labels = {
        'rest': 'Repos',
        'recovery': 'Récupération',
        'load': 'Charge'
    }
    for i, (_, row) in enumerate(phase_changes):
        left = pd.to_datetime(row['hires_tstart'], unit='s')
        right = pd.to_datetime(df_raw['ts'].max() if i == len(phase_changes) - 1 else phase_changes[i+1][1]['hires_tstart'], unit='s')
        fig.add_vrect(x0=left, x1=right, fillcolor=phase_colors[row['phase']], opacity=0.3, line_width=0, layer='below')
        fig.add_annotation(x=left, y=text_y, showarrow=False, text=phase_labels[row['phase']], xanchor='left', xshift=10)
    sighs = df_cycles[df_cycles['is_sigh']]
    keep_sigh_cols = [0, 1, 7, 14, 15, 16, 17, 35, 36, 38, 39]
    sighs = sighs.drop([sighs.columns[col] for col in range(len(sighs.columns)) if col not in keep_sigh_cols], axis=1)
    sighs.columns = ['Phase', 'Charge', 'Vol Courant', 'Vol Insp', 'T Insp', 'Vol Exp', 'T Exp', 'T start (s)', 'T end (s)', 'Durée (s)', 'Méd. courante Vol']
    print(f'{len(sighs)} soupirs')
    pd.set_option('display.max_rows', None)
    display(sighs)
    fig.show()

In [ ]:
select_display = widgets.Output()
plot_display = widgets.Output()
select = None
center_check = None
vol_select = None
def f():
    if not select or not center_check or not vol_select:
        return
    plot_display.clear_output()
    with plot_display:
        on_analyze_clicked(select.value, plot_display, sigh_vol=vol_select.value, rolling_center=center_check.value)

def on_upload_changed(inputs):
    global select
    global center_check
    global vol_select
    with select_display:
        select_display.clear_output()
        files = group_files(inputs['new'])
        center_check = widgets.Checkbox(value=True, description='Center point in rolling median window')
        select = widgets.Select(options=[(str(f.excel_path), f) for f in files])
        vol_select = widgets.Dropdown(options=[('Exp', 'ex'), ('Insp', 'in'), ('Average', 'avg')], description='Rolling vol', value='avg')
        button = widgets.Button(description='Analyze')
        button.on_click(lambda button: f())
        display(select)
        display(center_check)
        display(vol_select)
        display(button)

upload = widgets.FileUpload(
    multiple=True
)

upload.observe(on_upload_changed, names='value')

In [ ]:
upload

In [ ]:
display(select_display)
display(plot_display)